https://hackernoon.com/vector-databases-basics-of-vector-search-and-langchain-package-in-python


https://www.timescale.com/blog/a-python-library-for-using-postgresql-as-a-vector-database-in-ai-applications/

# LangChain FAISS

In [ ]:
!pip install langchain[all]
!pip3 install Langchain[FAISS]
!pip install faiss-cpu
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


In [ ]:
from langchain import FAISS
import pandas as pd
import numpy as np
import os
import time
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/DSM COURSE/NOTEBOOK/LLMs/Vector Database/labelled_newscatcher_dataset.csv", sep=";")
data

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en
...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en


In [ ]:
data.index.nunique()

RangeIndex(start=0, stop=108774, step=1)

# Convert the text data into vectors

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Metadata information

In [ ]:
start=time.time()
metadatas = []
for index, row in data.iterrows():
    doc_meta = {
        # "id": row['id'],
        "id": row.index,
        "language": row['lang']
    }
    metadatas.append(doc_meta)

# FAISS index

In [ ]:
faiss = FAISS.from_texts(data['title'].tolist(), embedding_function, metadatas)
print("Time Taken --> ", time.time()-start)

Time Taken -->  1229.673509836197


In [ ]:
save_path = "/content"
faiss.save_local(save_path, "dsm")

# Load the FAISS index with deserialization allowed
loaded_faiss = FAISS.load_local(
    save_path,
    embedding_function,
    "dsm",
    allow_dangerous_deserialization=True
)

# Top five similar documents t

In [ ]:
start=time.time()
loaded_faiss.similarity_search_with_score("Virus",5)

[(Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='New virus wave?'),
  0.72446346),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Letter: Skeptical of virus response - Letters'),
  0.7606702),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Quarterly update: A tale of 2 viruses'),
  0.7941258),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Another grim milestone in virus pandemic'),
  0.8267263),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Another grim milestone in virus pandemic'),
  0.8267263

In [ ]:
loaded_faiss.similarity_search_with_score("governors",5, {"language":"en"}, 10)

[(Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Governors find their moment — and peril'),
  0.5523976),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Ondo State governorship election: APC inaugurates national campaign council'),
  0.7943052),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='Why I want to be governor, explains Ajayi'),
  0.8353355),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'}, page_content='We don’t know who manages APC members’ list – Governors'),
  0.8439063),
 (Document(metadata={'id': Index(['topic', 'link', 'domain', 'published_date', 'title', 'lang'], dtype='object'), 'language': 'en'